In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [3]:
from sklearn.metrics import (
    accuracy_score, 
    f1_score, 
    classification_report, 
    confusion_matrix, 
    roc_auc_score, 
    precision_score, 
    recall_score,
)
from scikitplot.metrics import plot_roc
from scikitplot.metrics import plot_precision_recall

In [4]:
df = pd.read_csv("../our_analyses/dataset_prepared.csv")
df_test= pd.read_csv("../our_analyses/dataset_test_prepared.csv")

In [5]:
from sklearn.preprocessing import LabelEncoder

df=df.drop(['name', 'artists', 'album_name'], axis=1)
df_test=df_test.drop(['name', 'artists', 'album_name'], axis=1)

genre_groups = {
    'j-dance': 0, 'techno': 0, 'mandopop': 0, 'idm': 0, 'chicago-house': 0, 'breakbeat': 0,  # Elettronica/Dance
    'black-metal': 1,  # Metal/Rock
    'bluegrass': 2,  # Country/Folk
    'j-idol': 3, 'spanish': 3, 'disney': 3, 'brazil': 3,  # Pop/Commerciale
    'iranian': 4, 'afrobeat': 4, 'indian': 4, 'forro': 4,  # World
    'happy': 5, 'sleep': 5, 'study': 5,  # Ambient/New Age
    'industrial': 6  # Other
}

# Mappatura dei generi del dataframe ai gruppi di generi
df['genre_group'] = df['genre'].map(genre_groups).astype(int)
df_test['genre_group'] = df_test['genre'].map(genre_groups).astype(int)

# Stampa del mapping dei generi ai numeri corrispondenti
print(df['genre_group'].unique())

le = LabelEncoder()
df['explicit'] = le.fit_transform(df['explicit'])
df_test['explicit'] = le.fit_transform(df_test['explicit'])


[4 1 2 3 0 5 6]


In [6]:
X = df.values
y = np.array(df['genre_group'])
X = df.drop('genre_group', axis=1)
X = df.drop('genre', axis=1)

Z = df_test.values
t = np.array(df_test['genre_group'])
Z = df_test.drop('genre_group', axis=1)
Z = df_test.drop('genre', axis=1)

In [7]:
from sklearn.model_selection import train_test_split

random_state = 0

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=random_state
)

Z_train, Z_test, t_train, t_test = train_test_split(
    Z, t, test_size=0.4, stratify=t, random_state=random_state
)

In [15]:
from sklearn.preprocessing import StandardScaler

norm = StandardScaler()
norm = StandardScaler()
norm.fit(X_train)

X_train_norm = norm.transform(X_train)
X_test_norm = norm.transform(X_test)

norm.fit(Z_train)

Z_norm = norm.transform(Z)

Z_train_norm = norm.transform(Z_train)
Z_test_norm = norm.transform(Z_test)

In [16]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=14, metric="cityblock", weights="uniform")
clf.fit(X_train_norm, y_train)

y_test_pred = clf.predict(X_test_norm)
print("Accuracy:", accuracy_score(y_test, y_test_pred))


t_test_pred = clf.predict(Z_test_norm)
print("Accuracy:", accuracy_score(t_test, t_test_pred))

Accuracy: 0.9003333333333333
Accuracy: 0.8975


In [10]:
from sklearn.model_selection import learning_curve
import numpy as np
import matplotlib.pyplot as plt

# Genera la curva di apprendimento per il primo classificatore clf
train_sizes, train_scores, test_scores = learning_curve(
    estimator=clf,
    X=X_train_norm,
    y=y_train,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=10,
    scoring='accuracy',
    n_jobs=-1,
    shuffle=True,
    random_state=0
)

# Calcola la media e la deviazione standard per clf
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Genera la curva di apprendimento per il secondo classificatore clf_test
train_sizes_test, train_scores_test, test_scores_test = learning_curve(
    estimator=clf_test,
    X=Z_train_norm,
    y=t_train,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=10,
    scoring='accuracy',
    n_jobs=-1,
    shuffle=True,
    random_state=0
)

# Calcola la media e la deviazione standard per clf_test
train_mean_test = np.mean(train_scores_test, axis=1)
train_std_test = np.std(train_scores_test, axis=1)
test_mean_test = np.mean(test_scores_test, axis=1)
test_std_test = np.std(test_scores_test, axis=1)

# Disegna la curva di apprendimento
plt.figure(figsize=(10, 6))

# Curva per clf
plt.plot(train_sizes, train_mean, 'o-', color='blue', label='Training score (clf)')
plt.plot(train_sizes, test_mean, 'o-', color='green', label='Cross-validation score (clf)')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1, color='green')

# Curva per clf_test
plt.plot(train_sizes_test, train_mean_test, 's--', color='red', label='Training score (clf_test)')
plt.plot(train_sizes_test, test_mean_test, 's--', color='orange', label='Cross-validation score (clf_test)')
plt.fill_between(train_sizes_test, train_mean_test - train_std_test, train_mean_test + train_std_test, alpha=0.1, color='red')
plt.fill_between(train_sizes_test, test_mean_test - test_std_test, test_mean_test + test_std_test, alpha=0.1, color='orange')

# Aggiungi titolo e assegnazioni
plt.title('Learning Curve for k-NN Classifier')
plt.xlabel('Training Data Size')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.grid(True)
plt.show()


NameError: name 'clf_test' is not defined

In [ ]:
import plotly.graph_objs as go
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Assicurati che Z_train, Z_test, t_train, e t_test siano definiti e che i tuoi dati siano già preelaborati
# Z_train, Z_test, t_train, t_test = ...

# Previsione delle probabilità per il set di test
t_score = clf_test.predict_proba(Z_test_norm)

# Binarizza il vettore delle etichette in un formato "one-vs-all"
t_test_binarize = label_binarize(t_test, classes=np.unique(t_train))

# Calcola ROC curve e AUC per ciascuna classe
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = t_score.shape[1]

# Preparare i dati per il plot
traces = []

# Calcola la ROC curve per ogni classe
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(t_test_binarize[:, i], t_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    traces.append(go.Scatter(x=fpr[i], y=tpr[i], mode='lines', 
                             name=f'Class {i} (AUC = {roc_auc[i]:.2f})'))

# Aggiungi la linea diagonale che rappresenta il caso di "indovinare casualmente"
traces.append(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Chance', 
                         line=dict(dash='dash')))

# Crea il layout del plot
layout = go.Layout(title='Multiclass ROC Curve',
                   xaxis=dict(title='False Positive Rate'),
                   yaxis=dict(title='True Positive Rate'),
                   width=1000,
                   height=800,
                   showlegend=True,
                   legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig = go.Figure(data=traces, layout=layout)
fig.show()


In [ ]:
from sklearn.naive_bayes import GaussianNB, CategoricalNB

clf_gaussian = GaussianNB()
clf_gaussian.fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_pred
